In [2]:
!rm -r fashion-mnist
!unzip fashion-mnist.zip

Streaming output truncated to the last 5000 lines.
  inflating: fashion-mnist/test/0/2423.png  
  inflating: fashion-mnist/test/0/444.png  
  inflating: fashion-mnist/test/0/965.png  
  inflating: fashion-mnist/test/0/1460.png  
  inflating: fashion-mnist/test/0/3440.png  
  inflating: fashion-mnist/test/0/2795.png  
  inflating: fashion-mnist/test/0/2474.png  
  inflating: fashion-mnist/test/0/169.png  
  inflating: fashion-mnist/test/0/1647.png  
  inflating: fashion-mnist/test/0/2121.png  
  inflating: fashion-mnist/test/0/2163.png  
  inflating: fashion-mnist/test/0/2753.png  
  inflating: fashion-mnist/test/0/645.png  
  inflating: fashion-mnist/test/0/34.png  
  inflating: fashion-mnist/test/0/716.png  
  inflating: fashion-mnist/test/0/3874.png  
  inflating: fashion-mnist/test/0/3128.png  
  inflating: fashion-mnist/test/0/2180.png  
  inflating: fashion-mnist/test/0/2439.png  
  inflating: fashion-mnist/test/0/269.png  
  inflating: fashion-mnist/test/0/2027.png  
  inflating:

In [4]:
from glob import glob
train_files = glob('./fashion-mnist/train/*/*.png')
valid_files = glob('./fashion-mnist/valid/*/*.png')
test_files = glob('./fashion-mnist/test/*/*.png')

## Datos aleatorios

In [5]:
import numpy as np
np.random.shuffle(train_files)
np.random.shuffle(valid_files)
np.random.shuffle(test_files)
len(train_files), len(valid_files), len(test_files)

(55000, 10000, 5000)

### El modelo preentrenado que se usará es ResNet18, la cual es una CNN con 18 capas. La entrada de la red tiene un tamaño de 224x224x3, por lo cual se hará un resize 224x224 y se ampliaran los canales a 3 canales.

## Se reescalan los datos para adaptarlos a la entrada de 224x224 px con 3 canales (RGB) que recibe la ResNet18

In [3]:
import torchvision.transforms as transforms
data_transform = transforms.Compose([
                 transforms.Resize((224, 224)),
                 transforms.Grayscale(3), # Dado que fashion-mnist tiene un solo canal, lo cambiamos a 3 para no tener que modificar más capas en el modelo
                 transforms.ToTensor(),    
                 transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])   # ayuda a la red a converger mas rapido
                 ])

## Se organizan los datos en listas


In [ ]:
from PIL import Image   # lee imagenes como arrays

# se cargan los datos del train set
N_train = len(train_files)
X_train, Y_train = [], []

for i, train_file in enumerate(train_files):
  Y_train.append( int(train_file.split('/')[3]) )
  X_train.append( np.array(data_transform(Image.open(train_file) )))
  
# se cargan los datos del test set
N_test = len(test_files)
X_test, Y_test = [], []

for i, test_file in enumerate(test_files):
  Y_test.append( int(test_file.split('/')[3]) )
  X_test.append( np.array(data_transform(Image.open(test_file)) ))

np.shape(X_train)

## Se visualizan los datos

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,8))
for i in range(4):
  plt.subplot(2,2,i+1)
  plt.imshow(X_test[i*5].reshape(224,224,3))
  plt.title(Y_test[i*5])
  plt.axis(False)
plt.show()

## Convertir a tensores y batch size

In [ ]:
import torch
from torch.autograd import Variable

X_train = Variable(torch.from_numpy(np.array(X_train))).float()
Y_train = Variable(torch.from_numpy(np.array(Y_train))).long()

X_test = Variable(torch.from_numpy(np.array(X_test))).float()
Y_test = Variable(torch.from_numpy(np.array(Y_test))).long()

X_train.data.size()

In [ ]:
batch_size = 32

train_ds = torch.utils.data.TensorDataset(X_train, Y_train)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)

# Entrenando

In [ ]:
import torch

model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
model

In [ ]:
!pip install hiddenlayer

In [ ]:
import hiddenlayer as hl
hl.build_graph(model, torch.zeros([32,1,28,28]))